<a href="https://colab.research.google.com/github/Graziottomp/ImageFileConvertion/blob/main/Sentinel1nc2tiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!/usr/bin/env python
# coding: utf-8

# Rotina para converter arquivos Sentinel 1 em NetCDF para imagens GeoTiff:

#author = 'Maria Paula Graziotto'

# copyright = "Copyright 2023, Maria Paula Graziotto"

# credits = ['Graziotto, M. P.']

# license = "Public Domain"

# version = "4.0"

# maintainer = "Maria Paula Graziotto"

# email = 'graziotto.mp@outlook.com'
#--------------------------------------------------------------------------------------------------------

import pathlib as plib
import glob
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rio
import rasterio as rs
from rasterio.plot import show
from openpyxl import Workbook, load_workbook
from datetime import datetime
import locale
from pyproj import CRS, Transformer

def fsentinel(file2read):
    try:
        df_S1 = xr.open_dataset(file2read)
    except Exception as e:
        print(f'Error reading file: {file2read}\n{e}')
        return False

    # Extract metadata
    metadata = df_S1.metadata.attrs
    file_Id = 1
    file_path = str(plib.Path.cwd())
    file_name = metadata['Abstracted_Metadata:PRODUCT']
    file_indexed_by = 'INPE'
    file_type = 'NetCDF'
    img_projection = metadata['Abstracted_Metadata:geo_ref_system']
    img_bbox = [metadata[f'Abstracted_Metadata:first_{corner}']
                for corner in ['near_lat', 'near_long', 'far_lat', 'far_long']]
    img_res_lat = metadata['Abstracted_Metadata:lat_pixel_res']
    img_res_lon = metadata['Abstracted_Metadata:lon_pixel_res']
    img_inc_angle = metadata['Abstracted_Metadata:incidence_near']
    sat_name = metadata['Abstracted_Metadata:MISSION']
    sat_sensor = metadata['Abstracted_Metadata:ACQUISITION_MODE']
    sat_band = 'C'
    sat_orbit = metadata['Abstracted_Metadata:PASS']
    proc_level = metadata['Abstracted_Metadata:SPH_DESCRIPTOR']
    sat_proc_level = proc_level[14:25]
    sat_polar1 = metadata['Abstracted_Metadata:mds1_tx_rx_polar']
    sat_polar2 = metadata['Abstracted_Metadata:mds2_tx_rx_polar']
    sat_polar = f'{sat_polar1} + {sat_polar2}'

    # Extract date and time
    file_d = file2read[17:25]
    file_t = file2read[26:32]
    file_date_time = datetime.strptime(file_d + file_t, '%Y%m%d%H%M%S')

    # Save metadata to Excel
    bd_images = [file_Id, file_path, file_name, file_indexed_by, file_date_time,
                 file_type, img_projection] + img_bbox + [img_res_lat, img_res_lon,
                 img_inc_angle, sat_name, sat_sensor, sat_band, sat_orbit,
                 sat_proc_level, sat_polar]

    try:
        arquivo_excel = load_workbook('.csv')
        planilha2 = arquivo_excel.worksheets[1]
        planilha2.append(bd_images)
        arquivo_excel.save(".csv")
    except Exception as e:
        print(f'Error saving to Excel: {e}')

    # Raster conversion
    sig = df_S1['Sigma0_VV']
    lon = df_S1['lon']
    lat = df_S1['lat']

    sig = sig.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
    sig.rio.crs
    sig.rio.write_crs(4326, inplace=True)

    name = file2read.split(".")
    sig.rio.to_raster(f"{name[0]}.tiff")

    return True

def main():
    current_dir = plib.Path.cwd()
    arquivo_excel = load_workbook('.csv')
    nome_arquivos = glob.glob('*.nc')
    nome_arquivos.sort()

    for file2read in nome_arquivos:
        fsentinel(file2read)

if __name__ == "__main__":
    main()
